<small style="font-size:10px;">

# 📌 **Camada Prata**  

### **🔹 1️⃣ Importação de Bibliotecas**  
Carrega funções do **PySpark** para tratamento de strings, limpeza de dados e manipulação de colunas.  

### **🔹 2️⃣ Execução de Variáveis Globais**  
Executa o notebook `Variaveis`, que contém os caminhos do Data Lake.  

### **🔹 3️⃣ Leitura dos Dados da Camada Bronze**  
Carrega os dados brutos armazenados na **Camada Bronze (Delta Lake)** e exibe o schema.  

### **🔹 4️⃣ Tratamento de Valores Nulos**  
Substitui valores ausentes por padrões como `"Desconhecido"`, `"0.0"` e `"Sem_Site"`.  

### **🔹 5️⃣ Adição da Origem dos Dados**  
Cria a coluna `"fonte_dados"` para indicar que os dados vêm da **API Open Brewery DB**.  

### **🔹 6️⃣ Remoção de Colunas Desnecessárias**  
Exclui **`nm_estado_repetido` e `nm_rua`** por não serem relevantes.  

### **🔹 7️⃣ Criação de Novas Colunas**  
- `"Possui_Site?"` → **Identifica se há um site cadastrado**.  
- `"Esta_Ativa?"` → **Indica se a cervejaria está em operação**.  
- `"Localizacao"` → **Concatena país, estado e cidade para facilitar consultas**.  

### **🔹 8️⃣ Remoção de Duplicatas**  
Remove registros **com o mesmo nome, localização, endereço e CEP**.  

### **🔹 9️⃣ Padronização de CEP**  
Remove **traços e espaços** para manter um formato uniforme.  

### **🔹 🔟 Padronização de Nomes**  
Ajusta `"nm_cervejaria"` para **inicial maiúscula + remoção de espaços extras**.  

### **🔹 1️⃣1️⃣ Reordenação de Colunas**  
Organiza as colunas em uma estrutura **mais intuitiva** para consumo.  

### **🔹 1️⃣2️⃣ Particionamento e Salvamento**  
Os dados são **reparticionados por `Localizacao`** e salvos na **Camada Prata** em **Delta Lake** para otimização.  

---

✅ **Agora os dados estão limpos, padronizados e otimizados para a Camada Ouro!** 🚀  

</small>


In [0]:
from pyspark.sql.functions import when, concat, lit,regexp_replace,trim,initcap

In [0]:
%run "./Variaveis"  

In [0]:
df = spark.read.format("delta").load(f"{camada_bronze}cervejarias")
display(df)
df.printSchema()

In [0]:
df = df.fillna({
    "ds_endereco_1":"Desconhecido",
    "ds_endereco_2":"Desconhecido",
    "ds_endereco_3":"Desconhecido",
    "nr_longitude" :"0.0",
    "nr_latitude"  :"0.0",
    "nr_telefone" :"Desconhecido",
    "ds_site_web" : "Sem_Site",
})

In [0]:
df = df.withColumn("fonte_dados", lit("API Open Brewery DB"))


In [0]:
df_dropColumn = df.drop("nm_estado_repetido", "nm_rua")

In [0]:
df_possui_site = df_dropColumn.withColumn(
    "Possui_Site?", when(df_dropColumn.ds_site_web == "Sem_Site", False).otherwise(True)
)

In [0]:
df_esta_ativa = df_possui_site.withColumn(
    "Esta_Ativa?",
    when(df_possui_site.nm_tipo_cervejaria == "closed", False).otherwise(True),
)

In [0]:
df_localizacao = df_esta_ativa.withColumn("Localizacao",concat("nm_pais",lit(" - "),"nm_estado",lit(" - "),"nm_cidade"))

In [0]:
df_localizacao = df_localizacao.dropDuplicates(["nm_cervejaria","Localizacao","cd_cep","ds_endereco_1"])


In [0]:
df_cep = df_localizacao.withColumn("cd_cep", regexp_replace(df_localizacao.cd_cep,"-",""))
df_cep = df_cep.withColumn("cd_cep", regexp_replace(df_cep.cd_cep," ",""))


In [0]:
df_padronizar_nomeCervejaria = df_cep.withColumn("nm_cervejaria", trim(initcap(df_cep.nm_cervejaria)))


In [0]:
df_reordenar = df_padronizar_nomeCervejaria.select(
    "id_cervejaria",
    "nm_cervejaria",
    "nm_tipo_cervejaria",
    "Esta_Ativa?",
    "Localizacao",
    "ds_endereco_1",
    "ds_endereco_2",
    "ds_endereco_3",
    "nr_longitude",
    "nr_latitude",
    "nr_telefone",
    "Possui_Site?",
    "ds_site_web",
    "cd_cep",
    "nm_pais",
    "nm_estado",
    "nm_cidade",
    "fonte_dados",
    "data_ingestao"
)
df_reordenar.repartition(10,"Localizacao")
display(df_reordenar)


In [0]:
df_prata = (
    df_reordenar.write.format("delta")
    .mode("overwrite")
    .partitionBy("Localizacao")
    .save(f"{camada_prata}cervejarias")
)